In [23]:
import numpy as np
import pandas
import pyxdf

In [39]:
def xdf2csv(xdf_path, out_path, stream_names):
    '''
    Convert an xdf file to a csv file
    :param xdf_path: path to the xdf file
    :param out_path: path to the output csv file
    :param stream_names: dictionary containing the names of the streams to be extracted.
        Must contain the following keys:
            - 'filtered_data'
            - 'raw_data'
            - 'stimuli'
    :param channel: channel to be considered as the detection channel

    :return: a pandas dataframe containing the data
    '''
    # Load the xdf file
    print(f"Loading file {xdf_path}...")
    xdf_data, _ = pyxdf.load_xdf(xdf_path)
    print("Done.")

    # Load all streams given their names
    filtered_stream, raw_stream, markers = None, None, None
    for stream in xdf_data:
        # print(stream['info']['name'])
        if stream['info']['name'][0] == stream_names['filtered_data']:
            print(f"Found stream {stream['info']['name'][0]} | Length: {len(stream['time_stamps'])}")
            filtered_stream = stream
        elif stream['info']['name'][0] == stream_names['raw_data']:
            print(f"Found stream {stream['info']['name'][0]} | Length: {len(stream['time_stamps'])}")
            raw_stream = stream
        elif stream['info']['name'][0] == stream_names['stimuli']:
            print(f"Found stream {stream['info']['name'][0]} | Length: {len(stream['time_stamps'])}")
            markers = stream

    # We must have at least one of the streams
    assert filtered_stream is not None and raw_stream is not None, 'XDF ISSUE: At least one of Filtered or Raw data streams must be present in the XDF file.'
    
    # Both streams might not have the same exact length, so we need to find the minimum length
    if filtered_stream is None:
        shortest_stream_length = len(raw_stream['time_stamps'])
    elif raw_stream is None:
        shortest_stream_length = len(filtered_stream['time_stamps'])
    else:
        shortest_stream_length = min(len(filtered_stream['time_stamps']), len(raw_stream['time_stamps']))
    print(f"Using minimum length: {shortest_stream_length}")

    # Create a pandas dataframe
    df = pandas.DataFrame({
        'time_stamps': filtered_stream['time_stamps'][:shortest_stream_length]
    })

    # Checking that both streams have the right amount of channels
    if raw_stream is not None and filtered_stream is not None:
        assert filtered_stream['time_series'].shape[1] == raw_stream['time_series'].shape[1], 'XDF ISSUE: The number of channels in both streams must be the same.'

    # Get the number of channels
    if filtered_stream is not None:
        num_channels = filtered_stream['time_series'].shape[1]
    elif raw_stream is not None:
        num_channels = raw_stream['time_series'].shape[1]
    print(f"Found {num_channels} channels")

    # Add the data to the dataframe
    for i in range(num_channels):
        if filtered_stream is not None:
            df[f'filtered_data_{i}'] = filtered_stream['time_series'][:shortest_stream_length, i]
        if raw_stream is not None:
            df[f'raw_data_{i}'] = raw_stream['time_series'][:shortest_stream_length, i]

    # Add the stimuli to the dataframe if the stream is present
    if markers is not None:
        # Check that the markers are conform with the stream data
        assert min(markers['time_stamps']) >= min(df['time_stamps']), 'XDF ISSUE: All marker timestamps must be after the start of the data timestamps.'
        assert max(markers['time_stamps']) <= max(df['time_stamps']), 'XDF ISSUE: The marker timestamps must be before the end of the data timestamps.'

        # Find the closest index to each marker
        indexes = [np.argmin(np.abs(df['time_stamps'] - marker)) for marker in markers['time_stamps']]
        print(f"Found {len(indexes)} markers")

        # Create a vector of all zeros and set the indexes to 1
        stimuli = np.zeros(len(df['time_stamps'])).astype(int)
        stimuli[indexes] = 1

        # Add the stimuli to the dataframe
        df['stimuli'] = stimuli

    # Save the dataframe to a csv file
    if out_path is not None:
        print(f"Saving to file {out_path}...")
        df.to_csv(out_path, index=False)
        print("Done.")

    return df

In [40]:
STREAM_NAMES = {
    'filtered_data': 'Portiloop Filtered',
    'raw_data': 'Portiloop Raw Data',
    'stimuli': 'Portiloop_stimuli'
}
test_path = '/home/ubuntu/portiloop-training/temp/BSP_O20_Portiloop_LSL.xdf'

new_df = xdf2csv(test_path, './test_csv.csv', STREAM_NAMES)

Loading file /home/ubuntu/portiloop-training/temp/BSP_O20_Portiloop_LSL.xdf...
Done.
Found stream Portiloop_stimuli | Length: 86
Found stream Portiloop Raw Data | Length: 1893164
Found stream Portiloop Filtered | Length: 1893164
Using minimum length: 1893164
Found 8 channels
Found 86 markers
Saving to file ./test_csv.csv...
Done.


In [41]:
new_df.head()

,time_stamps,filtered_data_0,raw_data_0,filtered_data_1,raw_data_1,filtered_data_2,raw_data_2,filtered_data_3,raw_data_3,filtered_data_4,raw_data_4,filtered_data_5,raw_data_5,filtered_data_6,raw_data_6,filtered_data_7,raw_data_7,stimuli
0,290.103191,0.764521,6678.678711,-1.220135,37248.890625,-0.494485,33682.558594,-0.858977,41073.027344,-1.388191,35379.503906,0.0,0.0,0.0,0.0,0.0,0.0,0
1,290.107204,0.948249,6645.195801,-1.313397,37251.550781,-0.547019,33685.933594,-0.879827,41074.750000,-1.427210,35380.980469,0.0,0.0,0.0,0.0,0.0,0.0,0
2,290.111218,0.959185,6668.173340,-1.399982,37254.054688,-0.586612,33686.738281,-0.914044,41076.023438,-1.389168,35382.453125,0.0,0.0,0.0,0.0,0.0,0.0,0
3,290.115232,0.843993,6699.220215,-1.332869,37250.367188,-0.583396,33684.300781,-0.884762,41072.023438,-1.170882,35377.847656,0.0,0.0,0.0,0.0,0.0,0.0,0
4,290.119245,0.671473,6687.619629,-1.080098,37251.105469,-0.537073,33689.687500,-0.769400,41073.609375,-0.803061,35377.781250,0.0,0.0,0.0,0.0,0.0,0.0,0
